<a href="https://colab.research.google.com/github/kerryback/2022-638-binder/blob/main/7a-validation-with-industries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymssql


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from joblib import dump

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [2]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [3]:
data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m, siccd
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()
data['rnk'] = data.ret.rank(pct=True)

In [6]:
ff49 = pd.read_excel("files/ff49.xlsx", index_col=0)
def industry(sic):
  try:
    return ff49[(ff49.lower<=sic)&(sic<=ff49.upper)].index[0]
  except:
    return "Almost Nothing"
data["industry"] = data.siccd.map(industry)

In [10]:
transform1 = QuantileTransformer(
    output_distribution="normal"
)

transform2 = QuantileTransformer(
    output_distribution="normal"
)

poly = PolynomialFeatures(degree=2)

pipe1 = make_pipeline(transform1, poly, transform2)

onehot = OneHotEncoder()

preprocessor = make_column_transformer(
  (
    pipe1,
    ["roeq", "mom12m"]
  ),
  (
    onehot,
    ["industry"]
  )
)

model = RandomForestRegressor(
   random_state=0
)

pipe = make_pipeline(preprocessor, model)

In [11]:
param_grid = {
    "randomforestregressor__max_depth": [3, 4, 5]
}

cv = GridSearchCV(
  pipe, 
  param_grid=param_grid
)

X = data[["roeq", "mom12m", "industry"]]
y = data["rnk"]

cv.fit(X, y)

dump(cv, "forest3.joblib")

['forest3.joblib']